In [108]:
# %load notebook_setup.py
#Use %load notebook_setup.py to load these commands at the top of your
#ipython notebook.  Includes running the std_libraries.py file. 

%run std_libraries.py
%load_ext autoreload
%autoreload 2
# %matplotlib 
# plt.ioff()
#%matplotlib inline
%matplotlib notebook

Importing io_library.  If autoreload, may need to reset base_dir and data_processing dir 
  io_library.base_dir=base_dir 
 io_library.data_processing_dir = data_processing_dir
what computer are you on? a = Ben's laptop, b = gpucluster, c = Ben's desktop, d = otherb
base directory is /home/heineike/github/expression_broad_data
Importing expression plots and io_library and setting base_dir and data_processing_dir
are you online? Yes/No Yes
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from bs4 import BeautifulSoup


In [89]:
#Load GOI ohnolog data
fname = data_processing_dir + os.sep + os.path.normpath('kl_sc_PKA_as_m24_r1g1_20181017/gois_20181205.pkl')

with open(fname, 'rb') as file:
    [ohnologs_goi,ohnologs_expression_sorted,goi_criteria] = pickle.load(file)


In [ ]:
#Extract files using url
window = '8'

pillars_goi_fname = os.path.normpath(data_processing_dir + "ortholog_files_YGOB/YGOB_pillars_goi.txt")

with open(pillars_goi_fname,'w') as f:
    for anc in ohnologs_goi['Ancestor']:

        YGOB_query_url = 'http://ygob.ucd.ie/cgi/browser/tab.pl?ver=Latest&win=' + window + '&gene=' + anc
        YGOB_query_response = requests.get(YGOB_query_url)

        soup = BeautifulSoup(YGOB_query_response.text)
        table = soup.find("table")

        output_rows = []
        for table_row in table.findAll('tr'):
            columns = table_row.findAll('td')
            output_row = []
            for column in columns:
                output_row.append(column.text)
            output_rows.append(output_row)

        synteny_window = pd.DataFrame(output_rows[2:],columns = output_rows[1])
        synteny_row = synteny_window[synteny_window['Ancestor']==anc]
        print("\t".join(synteny_row.iloc[0,:].values) + '\n')
        f.write("\t".join(synteny_row.iloc[0,:].values) + '\n')

In [110]:
#Build ortholog mapping from S.Cer to post WGH species for gois
spec_order_post_WGH = ['Scer','Smik','Skud','Suva', 'Cgla', 'Kafr','Knag','Ndai','Ncas','Tbla','Tpha','Vpol'] 

spec1 = 'Scer'
for spec2 in list(set(spec_order_post_WGH)-set(spec1)):
    io_library.write_YGOB_orth_lookup_table_goi(spec1,spec2)

/home/heineike/github/expression_broad_data/expression_data/ortholog_files_YGOB/Scer-Ndai-goi-orthologs.txt
/home/heineike/github/expression_broad_data/expression_data/ortholog_files_YGOB/Scer-Vpol-goi-orthologs.txt
/home/heineike/github/expression_broad_data/expression_data/ortholog_files_YGOB/Scer-Kafr-goi-orthologs.txt
/home/heineike/github/expression_broad_data/expression_data/ortholog_files_YGOB/Scer-Knag-goi-orthologs.txt
/home/heineike/github/expression_broad_data/expression_data/ortholog_files_YGOB/Scer-Cgla-goi-orthologs.txt
/home/heineike/github/expression_broad_data/expression_data/ortholog_files_YGOB/Scer-Scer-goi-orthologs.txt
/home/heineike/github/expression_broad_data/expression_data/ortholog_files_YGOB/Scer-Tbla-goi-orthologs.txt
/home/heineike/github/expression_broad_data/expression_data/ortholog_files_YGOB/Scer-Ncas-goi-orthologs.txt
/home/heineike/github/expression_broad_data/expression_data/ortholog_files_YGOB/Scer-Skud-goi-orthologs.txt
/home/heineike/github/expres

In [107]:
spec1= 'Scer'
spec2= 'Smik'
#def write_YGOB_orth_lookup_table_goi(spec1, spec2):
#This is necessary because pillars does not specify syntenic assignment for post-WGH genes
#uses data generated from YGOB in 20190708_DEpka_post_WGH_YGOB_synteny_assign

#Makes an orhtolog lookup table for all species in the YGOB Pillars data for just goi
#spec1 the index, spec2 is the lookup.  multiple orthologs separated by tabs, NONE if there are none.  

#Note that this cannot lookup Small Scale Duplications after the WGH because that data is not part of the pillars 

fname = os.path.normpath(data_processing_dir + "ortholog_files_YGOB/YGOB_pillars_goi.txt")

    #YGOB_Pillars.txt order of species: 
    #    0    V. polyspora Position B
    #    1    T. phaffii Position B
    #    2    T. blattae Position B
    #    3    N. dairenensis Position B
    #    4    N. castellii Position B
    #    5    K. naganishii Position B
    #    6    K. africana Position B
    #    7    C. glabrata Position B
    #    8    S. bayanus var. uvarum Position B
    #    9    S. kudriavzevii Position B
    #    10   S. mikatae Position B
    #    11   S. cerevisiae Position B
    #    12   L. waltii
    #    13   L. thermotolerans
    #    14   L. kluyveri
    #    15   E. cymbalariae
    #    16   E. gossypii 
    #    17   K. lactis
    #    18   T. delbrueckii
    #    19   Z. rouxii
    #    20   Ancestral Gene Order
    #    21   S. cerevisiae Position A
    #    22   S. mikatae Position A
    #    23   S. kudriavzevii Position A
    #    24   S. bayanus var. uvarum Position A
    #    25   C. glabrata Position A
    #    26   K. africana Position A
    #    27   K. naganishii Position A
    #    28   N. castellii Position A
    #    29   N. dairenensis Position A
    #    30   T. blattae Position A
    #    31   T. phaffii Position A
    #    32   V. polyspora Position A

    #position in pillars file of ancestor
    #anc_pos = 20

    #for regev lab expression data, Suva and Sbay have same gene names. 
    orth_positions_post_WGH = {'Vpol': [0,32], 'Tpha': [1,31], 'Tbla': [2,30], 'Ndai': [3,29], 
                      'Ncas': [4,28], 'Knag': [5,27], 'Kafr': [6,26], 'Cgla': [7,25],
                      'Suva': [8,24], 'Sbay':[8,24], 'Skud': [9,23], 'Smik': [10,22], 'Scer' : [11,21]}


    orth_positions_pre_WGH = {'Zrou':19, 'Tdel':18, 'Klac':17, 'Egos':16, 'Ecym': 15, 'Lklu':14, 'Lthe':13, 'Lwal':12}

    #Case1: spec1 post WGH, spec2 pre WGH

    if ((spec1 in orth_positions_post_WGH.keys()) & (spec2 in orth_positions_pre_WGH.keys())): 
        spec1_columns = orth_positions_post_WGH[spec1]
        spec2_column = orth_positions_pre_WGH[spec2]

        with open(fname) as f:
            orth_lookup = {}
            for line in f:
                linesp = line.split()
                for spec1_column in spec1_columns: 
                    spec1_gene = linesp[spec1_column]
                    if spec1_gene!= '---':
                        spec2_gene = linesp[spec2_column]
                        if spec2_gene!='---': 
                            orth_lookup[spec1_gene] = spec2_gene
                        else: 
                            orth_lookup[spec1_gene] = 'NONE'

    #Case2: spec1 pre WGH, spec2 pre WGH

    if ((spec1 in orth_positions_pre_WGH.keys()) & (spec2 in orth_positions_pre_WGH.keys())): 
        spec1_column = orth_positions_pre_WGH[spec1]
        spec2_column = orth_positions_pre_WGH[spec2]

        with open(fname) as f:
            orth_lookup = {}
            for line in f:
                linesp = line.split()
                spec1_gene = linesp[spec1_column]
                if spec1_gene!= '---':
                    spec2_gene = linesp[spec2_column]
                    if spec2_gene!='---': 
                        orth_lookup[spec1_gene] = spec2_gene
                    else: 
                        orth_lookup[spec1_gene] = 'NONE'




    #Case 3: spec1 pre WGH, spec2 post WGH
    #In this case there will often be more than one ortholog per pre WGH gene

    if ((spec1 in orth_positions_pre_WGH.keys()) & (spec2 in orth_positions_post_WGH.keys())): 
        spec1_column = orth_positions_pre_WGH[spec1]
        spec2_columns = orth_positions_post_WGH[spec2]

        with open(fname) as f:
            orth_lookup = {}
            for line in f:
                linesp = line.split()
                spec1_gene = linesp[spec1_column]
                if spec1_gene!= '---':
                    spec2_genes = []
                    for spec2_column in spec2_columns: 
                        spec2_gene = linesp[spec2_column]
                        if spec2_gene!='---':
                            spec2_genes.append(spec2_gene)
                    if len(spec2_genes)==0:
                        orth_lookup[spec1_gene] = 'NONE'
                    else: 
                        orth_lookup[spec1_gene] = spec2_genes


    #Case 4: spec1 pre WGH, spec2 pre WGH
    #use tracks in pillars file to match up WGH genes.  If there are two paralogs, this will match tracks. 
    #If there is one paralog, and there are two orthologs, it will list both.  
    #If there is only one ortholog for one gene, it will list the other one regardless of track.  

    if ((spec1 in orth_positions_post_WGH.keys()) & (spec2 in orth_positions_post_WGH.keys())): 
        spec1_columns = orth_positions_post_WGH[spec1]
        spec2_columns = orth_positions_post_WGH[spec2]

        with open(fname) as f:
            orth_lookup = {}
            for line in f:
                linesp = line.split()
                spec1_genes = []
                for spec1_column in spec1_columns: 
                    spec1_gene = linesp[spec1_column]
                    if spec1_gene!= '---':
                        spec1_genes.append(spec1_gene)

                if len(spec1_genes)==2: 
                    for jj,spec1_gene in enumerate(spec1_genes): 
                        spec2_column = spec2_columns[jj]
                        spec2_gene = linesp[spec2_column]
                        if spec2_gene!='---':
                            orth_lookup[spec1_gene] = [spec2_gene]
                        else: 
                            orth_lookup[spec1_gene] = ['NONE']

                elif len(spec1_genes)==1: 
                    spec1_gene = spec1_genes[0]
                    spec2_genes = []
                    for spec2_column in spec2_columns: 
                        spec2_gene = linesp[spec2_column]
                        if spec2_gene!='---':
                            spec2_genes.append(spec2_gene)
                    if len(spec2_genes)==0:
                        orth_lookup[spec1_gene] = 'NONE'
                    else: 
                        orth_lookup[spec1_gene] = spec2_genes





    orth_lookup_outputfname = os.path.normpath(data_processing_dir + 'ortholog_files_YGOB/' + spec1 + "-" + spec2 + "-goi-orthologs.txt"  )

    print(orth_lookup_outputfname)
    io_library.print_ortholog_file(orth_lookup_outputfname, orth_lookup)

    return orth_lookup 

/home/heineike/github/expression_broad_data/expression_data/ortholog_files_YGOB/Scer-Smik-goi-orthologs.txt
